In [14]:
import sys
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier

In [4]:
def df_to_list(dataframe):
    ## Outputs to three lists ankle, hand, pocket and the tester 
    ## Assumed: The dataframe is for the same tester and it is ordered by 3 ankles, 3 hands, 3 pockets
    total_list = dataframe[['x_accel', 'y_accel', 'z_accel']].values.tolist()
    
    ankle = total_list[0:3]
    hand = total_list[3:6]
    pocket = total_list[6:9]
    tester = dataframe['tester'].tolist()[0:3] # array of size 3 so easy to plug into GaussianNB
    
    return ankle, hand, pocket, tester

In [5]:
def full_df_to_list(dataframe):
    ## Outputs full list regardless of where data was recorded
    ## Assumed: The dataframe is for the same tester
    total_list = dataframe[['x_accel', 'y_accel', 'z_accel']].values.tolist()
    tester = dataframe['tester'].tolist()
    
    return total_list, tester

In [6]:
def gaussian(x, y):
    model = GaussianNB()
    x_train, x_valid, y_train, y_valid = train_test_split(x, y)
    model.fit(x_train, y_train)
    return model.score(x_valid, y_valid)

In [7]:
def polyfit(x, y):
    model = make_pipeline(
        PolynomialFeatures(degree=9, include_bias=True),
        LinearRegression(fit_intercept=False)
    )
    x_train, x_valid, y_train, y_valid = train_test_split(x, y)
    model.fit(x_train, y_train)
    return model.score(x_valid, y_valid)

In [9]:
def neighbors(x, y):
    model = make_pipeline(
    StandardScaler(),
    KNeighborsClassifier(n_neighbors=9)
    )
    x_train, x_valid, y_train, y_valid = train_test_split(x, y)
    model.fit(x_train, y_train)
    return model.score(x_valid, y_valid)

In [10]:
def decision(x, y):
    model = DecisionTreeClassifier(max_depth=8)
    x_train, x_valid, y_train, y_valid = train_test_split(x, y)
    model.fit(x_train, y_train)
    return model.score(x_valid, y_valid)

In [12]:
def svc(x, y):
    model = make_pipeline(
    PCA(250),
    SVC(kernel='linear', C=2.0)
    )
    x_train, x_valid, y_train, y_valid = train_test_split(x, y)
    model.fit(x_train, y_train)
    return model.score(x_valid, y_valid)

In [26]:
def mlp(x, y):
    model = MLPClassifier(solver='lbfgs', hidden_layer_sizes=())
    x_train, x_valid, y_train, y_valid = train_test_split(x, y)
    model.fit(x_train, y_train)
    return model.score(x_valid, y_valid)

In [46]:
p1_data_mean = pd.read_csv('p1-mean.csv')
p1_data_std = pd.read_csv('p1-std.csv')
p2_data_mean = pd.read_csv('p2-mean.csv')
p2_data_std = pd.read_csv('p2-std.csv')
p3_data_mean = pd.read_csv('p3-mean.csv')
p3_data_std = pd.read_csv('p3-std.csv')
p4_data_mean = pd.read_csv('p4-mean.csv')
p4_data_std = pd.read_csv('p4-std.csv')

left_data_mean = pd.read_csv('left-mean.csv')
left_data_std = pd.read_csv('left-std.csv')
right_data_mean = pd.read_csv('right-mean.csv')
right_data_std = pd.read_csv('right-std.csv')

In [48]:
# Holds different values per where the phone recorded it
list_mean = [p1_data_mean, p2_data_mean, p3_data_mean, p4_data_mean]
list_std = [p1_data_std, p2_data_std, p3_data_std, p4_data_std]
ankle_mean = []; hand_mean = []; pocket_mean = []; tester = []
ankle_std = []; hand_std = []; pocket_std = []

for item in list_mean:
    a, h, p, t = df_to_list(item)
    ankle_mean.append(a)
    hand_mean.append(h)
    pocket_mean.append(p)
    tester_mean.append(t)
    
for item in list_std:
    a, h, p, t = df_to_list(item)
    ankle_std.append(a)
    hand_std.append(h)
    pocket_std.append(p)